# Getting Started

This is a simple example showing:
1. Calling the [`market-data/v3/value/current/symbol` endpoint](https://developer.platts.com/servicecatalog#/MarketData/v3/Market%20Data) with an access token and api key
2. Putting the response into a DataFrame

## Fetching Data

### Install Dependencies

```{tip}
`pip install requests pandas pprint`
```

Import dependencies

In [1]:
import requests
import pandas as pd
from pprint import pprint


```{seealso}
This `get_token` function is taken from [Generating a Token](../token.ipynb)
```

In [2]:
def get_token(username, password, apikey):
  body = {
    "username": username,
    "password": password
  }
  headers = {
    "appkey": apikey
  }
  try:
    r = requests.post("https://api.platts.com/auth/api", data=body, headers=headers)
    r.raise_for_status()
    return r.json()["access_token"]
  except Exception as err:
    if r.status_code >= 500:
      print(err)
    else:
      print(r.status_code, r.json())

### Get Current Assessments

1) Create a function - `get_current_assessments` - to call the Platts Current Endpoint
2) Requires `username`, `password`, `apikey` and a list of `symbols`
3) Returns a `dict` (json) of the data from the Platts API

In [3]:
def get_current_assessments(username, password, apikey, symbols):
  token = get_token(username, password, apikey)

  # quotes are required around each symbol
  symbols = [ '"' + x + '"' for x in symbols]

  params = {
    "filter": f"symbol in ({','.join(symbols)})"
  }
  headers = {
    "Authorization": f"Bearer {token}",
    "appkey": apikey
  }

  try:
    r = requests.get("https://api.platts.com/market-data/v3/value/current/symbol", 
      params=params, headers=headers)
    r.raise_for_status()
    return r.json()
  except Exception as err:
    if r.status_code >= 500:
      print (err)
    else:
      print(r.status_code, r.json())
    raise



Invoke `get_current_assessments` and print results

In [4]:
symbols = ["PCAAS00", "PCAAT00"]
data = get_current_assessments("USERNAME", "PASSWORD", "APIKey", symbols)
pprint(data["results"])

[{'data': [{'assessDate': '2022-06-10T00:00:00',
            'bate': 'l',
            'isCorrected': 'N',
            'modDate': '2022-06-10T18:00:47',
            'value': 127.54},
           {'assessDate': '2022-06-10T00:00:00',
            'bate': 'h',
            'isCorrected': 'N',
            'modDate': '2022-06-10T18:00:47',
            'value': 127.56},
           {'assessDate': '2022-06-10T00:00:00',
            'bate': 'c',
            'isCorrected': 'N',
            'modDate': '2022-06-10T18:00:47',
            'value': 127.55}],
  'symbol': 'PCAAS00'},
 {'data': [{'assessDate': '2022-06-10T00:00:00',
            'bate': 'c',
            'isCorrected': 'N',
            'modDate': '2022-06-10T08:40:35',
            'value': 118.95},
           {'assessDate': '2022-06-10T00:00:00',
            'bate': 'h',
            'isCorrected': 'N',
            'modDate': '2022-06-10T08:40:35',
            'value': 118.96},
           {'assessDate': '2022-06-10T00:00:00',
            'bat

## Using Pandas

[ Pandas ]( https://pandas.pydata.org/ ) is one of the best tools for data anlysis in Python and converting our data into a Data Frame is necessary in order to use it so let's do that now.

### Creating a DataFrame
Flattening our JSON response into a DataFrame

In [5]:
df = pd.json_normalize(data['results'], record_path=['data'], meta="symbol")
# Parse dates so that we can use datetime operations
cols = ["assessDate", "modDate"]
df[cols] = df[cols].apply(pd.to_datetime)
df

,bate,assessDate,value,isCorrected,modDate,symbol
0,l,2022-06-10,127.54,N,2022-06-10 18:00:47,PCAAS00
1,h,2022-06-10,127.56,N,2022-06-10 18:00:47,PCAAS00
2,c,2022-06-10,127.55,N,2022-06-10 18:00:47,PCAAS00
3,c,2022-06-10,118.95,N,2022-06-10 08:40:35,PCAAT00
4,h,2022-06-10,118.96,N,2022-06-10 08:40:35,PCAAT00
5,l,2022-06-10,118.94,N,2022-06-10 08:40:35,PCAAT00


### Ensuring Datatypes are Correct
`assessDate` and `modDate` should be dates not objects

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   bate         6 non-null      object        
 1   assessDate   6 non-null      datetime64[ns]
 2   value        6 non-null      float64       
 3   isCorrected  6 non-null      object        
 4   modDate      6 non-null      datetime64[ns]
 5   symbol       6 non-null      object        
dtypes: datetime64[ns](2), float64(1), object(3)
memory usage: 416.0+ bytes


### Simple Pandas Example

Filtering to only "Close" prices 

In [7]:
filt = df["bate"] == "c"
close_df = df[filt]
close_df

,bate,assessDate,value,isCorrected,modDate,symbol
2,c,2022-06-10,127.55,N,2022-06-10 18:00:47,PCAAS00
3,c,2022-06-10,118.95,N,2022-06-10 08:40:35,PCAAT00
